
Imports

In [129]:
import tensorflow as tf
from tensorflow import keras
import time

Carregamento das funções que nos darão os modelos para cada topologia de rede.

In [130]:
# Crie o modelo de rede neural convolucional simples
def get_cifar10_network(num_classes,input_shape):
    model = keras.Sequential([
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),#(32, 32, 3) porque as imagens são 32X32 e RGB, portanto, tendo 3 canais de cor
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(num_classes, activation='softmax')  # 10 classes de saída
    ])

    # Compile o modelo
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',#pode ser substituída pela esparse_categorical_cross_entropy
                  metrics=['accuracy'])

    model.summary()

    return model

def get_fashion_mnist_network(num_classes,input_shape):
  model = keras.Sequential([
      tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),#(32, 32, 3) porque as imagens são 32X32 e RGB, portanto, tendo 3 canais de cor
      tf.keras.layers.MaxPooling2D((2, 2)),
      tf.keras.layers.Conv2D(128, (3, 3), activation='relu', input_shape=input_shape),  # Aumentando para 64 filtros
      tf.keras.layers.MaxPooling2D((2, 2)),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(64, activation='relu'),
      tf.keras.layers.Dense(num_classes, activation='softmax')  # 10 classes de saída
  ])

  # Compile o modelo
  model.compile(optimizer='adam',
                loss='categorical_crossentropy',#pode ser substituída pela esparse_categorical_cross_entropy
                metrics=['accuracy'])

  model.summary()

  return model
  #pass  # substitua pelo seu codigo

def get_mnist_network(num_classes,input_shape):
  model = keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),  # Usando 32 filtros
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),  # Aumentando para 64 filtros
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='softmax') 
  ])

  # Compile o modelo
  model.compile(optimizer='adam',
                loss='categorical_crossentropy',#pode ser substituída pela esparse_categorical_cross_entropy
                metrics=['accuracy'])

  model.summary()

  return model

def get_cifar100_network(num_classes,input_shape):
  model = keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),#(32, 32, 3) porque as imagens são 32X32 e RGB, portanto, tendo 3 canais de cor
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', input_shape=input_shape),  # Aumentando para 64 filtros
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu', input_shape=input_shape),  # Aumentando para 64 filtros
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='softmax')  # 10 classes de saída
  ])

  # Compile o modelo
  model.compile(optimizer='adam',
                loss='categorical_crossentropy',#pode ser substituída pela esparse_categorical_cross_entropy
                metrics=['accuracy'])

  model.summary()

  return model

In [131]:
model_functions = [get_mnist_network, get_fashion_mnist_network, get_cifar10_network, get_cifar100_network]

In [132]:
topologies = [
    {
        "name": 'mnist',
        "input_shape": (28,28,1),
        "num_classes": 10
    },
    {
        "name": 'fashion_mnist',
        "input_shape": (28,28,1),
        "num_classes": 10
    },
    {
        "name": 'cifar10',
        "input_shape": (32,32,3),
        "num_classes": 10
    },
    {
        "name": 'cifar100',
        "input_shape": (32,32,3),
        "num_classes": 100
    }
]

Carregamento de cada dataset e treinamento

In [133]:
#deixando a classe de
results = {}

In [134]:
# Carregar o conjunto de dados CIFAR-10
for tp in topologies:
    print("-----------------------")
    print(f"using {tp['name']} dataset")
    dataset = getattr(keras.datasets, tp['name'])
    results[tp['name']] = {}
    for function in model_functions:
        print(f"training using {function.__name__} model with {tp['name']} dataset\n")    
        #Carrega duas tuplas, representando os dados de treinamento e de teste.
        #Cada tupla tem as imagens e os respectivos rótulos
        (train_images, train_labels), (test_images, test_labels) = dataset.load_data()
        num_classes = tp['num_classes']
        # Converter para codificação one-hot dos labels
        train_labels = tf.keras.utils.to_categorical(train_labels, num_classes=num_classes)
        test_labels = tf.keras.utils.to_categorical(test_labels, num_classes=num_classes)
        #Não é necessário se utilizar como função de custo esparse_categorical_cross_entropy

        model = function(num_classes=num_classes, input_shape=tp['input_shape'])
        
        inicio = time.time()
        model.fit(train_images, train_labels, epochs=10)
        fim = time.time()

        # Avalie o modelo no conjunto de teste
        test_loss, test_accuracy = model.evaluate(test_images, test_labels)
        print(f'test accuracy: {test_accuracy * 100:.2f}%')
        
        results[tp['name']][f'{function.__name__}'] = {'time': (fim - inicio), 'acc': f'{test_accuracy * 100:.2f}%'}
        print("finishing training")
        print("~~~~~~~~~~~~~~~~~~")
    print("finishing using dataset")
    print("-----------------------\n")
    del dataset

-----------------------
using mnist dataset
training using get_mnist_network model with mnist dataset



Model: "sequential_45"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_103 (Conv2D)             │ (None, 26, 26, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_103               │ (None, 13, 13, 32)     │             0 │
│ (MaxPooling2D)                  │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_104 (Conv2D)             │ (None, 11, 11, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_104               │ (None, 5, 5, 64)       │             0 │
│ (MaxPooling2D)                  │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_52 (Flatten)            │ (None, 1600)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_104 (Dense)               │ (None, 64)             │       102,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_105 (Dense)               │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 121,930 (476.29 KB)

 Trainable params: 121,930 (476.29 KB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_132 (Conv2D)             │ (None, 30, 30, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_132               │ (None, 15, 15, 32)     │             0 │
│ (MaxPooling2D)                  │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_133 (Conv2D)             │ (None, 13, 13, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_133               │ (None, 6, 6, 64)       │             0 │
│ (MaxPooling2D)                  │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_134 (Conv2D)             │ (None, 4, 4, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_134               │ (None, 2, 2, 128)      │             0 │
│ (MaxPooling2D)                  │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_67 (Flatten)            │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_134 (Dense)               │ (None, 64)             │        32,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_135 (Dense)               │ (None, 100)            │         6,500 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 132,580 (517.89 KB)

 Trainable params: 132,580 (517.89 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 12s 7ms/step - accuracy: 0.0105 - loss: 5.1535
Epoch 2/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 11s 7ms/step - accuracy: 0.0092 - loss: 4.6059
Epoch 3/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 11s 7ms/step - accuracy: 0.0092 - loss: 4.6057
Epoch 4/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 11s 7ms/step - accuracy: 0.0091 - loss: 4.6030
Epoch 5/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 11s 7ms/step - accuracy: 0.0394 - loss: 4.3608
Epoch 6/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 11s 7ms/step - accuracy: 0.0966 - loss: 3.9121
Epoch 7/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 11s 7ms/step - accuracy: 0.1335 - loss: 3.6810
Epoch 8/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 11s 7ms/step - accuracy: 0.1622 - loss: 3.5149
Epoch 9/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 11s 7ms/step - accuracy: 0.1870 - loss: 3.3745
Epoch 10/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 11s 7ms/step - accuracy: 0.2007 - loss: 3.2905
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.1740 - loss: 3.5204
test accuracy: 17.32%
finishing tr

Preencha o dict abaixo substituindo os None com a acuracia final (acc) e o tempo de treinamento (time) encontrado no seu experimento pra cada dataset

In [135]:
results

{'mnist': {'get_mnist_network': {'time': 87.86078691482544, 'acc': '98.82%'},
  'get_fashion_mnist_network': {'time': 128.77173948287964, 'acc': '98.65%'},
  'get_cifar10_network': {'time': 75.56523895263672, 'acc': '97.94%'},
  'get_cifar100_network': {'time': 102.12647199630737, 'acc': '98.62%'}},
 'fashion_mnist': {'get_mnist_network': {'time': 87.324049949646,
   'acc': '89.03%'},
  'get_fashion_mnist_network': {'time': 120.38419651985168, 'acc': '89.55%'},
  'get_cifar10_network': {'time': 75.54393124580383, 'acc': '89.12%'},
  'get_cifar100_network': {'time': 102.45778608322144, 'acc': '88.02%'}},
 'cifar10': {'get_mnist_network': {'time': 92.44230556488037, 'acc': '66.35%'},
  'get_fashion_mnist_network': {'time': 147.06519412994385, 'acc': '65.41%'},
  'get_cifar10_network': {'time': 83.75397777557373, 'acc': '10.00%'},
  'get_cifar100_network': {'time': 128.82412815093994, 'acc': '65.73%'}},
 'cifar100': {'get_mnist_network': {'time': 93.96273279190063, 'acc': '1.00%'},
  'get

In [138]:
a = {}
a['oi'] = {}
a['oi']["fashionmnist"] = 10
a

{'oi': {'fashionmnist': 10}}